## Imports

In [21]:
import sys, os
import numpy as np
import pandas as pd
import pretty_midi as pm

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

from src.io.midi_input import MidiInput
from src.io import output

## Ratings

In [22]:
stability = (6, 2, 4, 2, 5, 4, 2, 5, 2, 4, 2, 4, 6) # index = pitch relative to root note
proximity = (24, 36, 32, 25, 20, 16, 12, 9, 6, 4, 2, 1, 0.25) # index = interval in semitones; mobility factor (* 2/3 for repetition) is already included at index 0
direction_continuation = (6, 20, 12, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0) # index = previous interval in semitones
direction_reversal = (0, 0, 0, 0, 0, 6, 12, 25, 36, 52, 75, 75, 75) # index = previous interval in semitones

## Expectancy Formula

In [23]:
def calc_expectancy_rating_for_pitch(candidate: int, prev: int, prevprev: int, single_ratings = False):
    
    prev_interval = abs(prev-prevprev)
    current_interval = abs(candidate-prev)
    prev_direction = np.sign(prev-prevprev)
    current_direction = np.sign(candidate-prev)

    continuation = current_direction != 0 and current_direction == prev_direction

    s = stability[candidate]
    p = proximity[current_interval]
    d = direction_continuation[prev_interval] if continuation else direction_reversal[prev_interval]

    if current_direction == 0:
        d /= 3

    if prev == 11 and candidate == 9:
        s = 6

    if single_ratings:
        return s, p, d
    else:
        return (s * p) + d

## Record Starting Notes

In [24]:
input = MidiInput()

IO: "MPK Mini Mk II 0" connected as input port


In [25]:
midi = input.recordNotes(2)
notes = midi.instruments[0].notes
input.close_port()

IO: NOW RECORDING ...
65 note off event
69 note off event

IO: recording successful.
IO: Input port "MPK Mini Mk II 0" closed.


## Prediction Settings

In [26]:
number_of_generated_notes = 8
root_offset = 60
randomized_choice = False

## Normalize Starting Notes

As no rhythm and articulation will be generated, all generated notes will have a velocity of *100* and a length of *0.5 seconds*.

For the sake of consistency, the input notes will be normalized to the same values.

Pitch values are shifted so that the root note has a pitch of 0.

In [27]:
for note in notes:
    note.velocity = 100
    note.pitch -= root_offset
    note.start = notes.index(note)/2
    note.end = note.start+0.5

In [28]:
# debug

notes

[Note(start=0.000000, end=0.500000, pitch=5, velocity=100),
 Note(start=0.500000, end=1.000000, pitch=9, velocity=100)]

## Calculation of Expectancy Ratings and Prediction

In [29]:
expectancy_ratings = []
predictions = []


for note in range(0, number_of_generated_notes):
    prev_pitch = notes[-1].pitch
    prevprev_pitch = notes[-2].pitch
    ratings_for_note = []

    # calculate ratings for all 12 possible pitches
    for pitch in range(0,13):
        e = calc_expectancy_rating_for_pitch(pitch, prev_pitch, prevprev_pitch)
        ratings_for_note.append(e)

    # append expectancy ratings to list
    expectancy_ratings.append(ratings_for_note)

    # select pitch with highest rating
    if not randomized_choice:
        prediction = ratings_for_note.index(max(ratings_for_note))
    else:
        normalized_ratings = [float(rating)/sum(ratings_for_note) for rating in ratings_for_note]
        prediction = np.random.choice(range(0,13), 1, p=normalized_ratings)[0]

    predictions.append(prediction)
    print('prediction ' + str(note) + ': ' + str(prediction))

    # create and append predicted note to midi
    predicted_note = pm.Note(100, prediction, note/2+1, note/2+1.5)
    notes.append(predicted_note)

prediction 0: 7
prediction 1: 5
prediction 2: 4
prediction 3: 2
prediction 4: 0
prediction 5: 0
prediction 6: 0
prediction 7: 0


In [30]:
table = pd.DataFrame(data=expectancy_ratings)
table['prediction'] = predictions
table

,0,1,2,3,4,5,6,7,8,9,10,11,12,prediction
0,24.0,12,36.0,24,80.0,80.0,50,160.0,72,96.0,72,128,150.0,7
1,66.0,36,76.0,52,137.0,140.0,84,120.0,72,128.0,50,80,96.0,5
2,108.0,52,112.0,76,192.0,96.0,72,160.0,50,80.0,32,48,54.0,4
3,140.0,70,148.0,92,120.0,144.0,64,125.0,40,64.0,24,36,36.0,2
4,204.0,84,96.0,72,160.0,100.0,40,80.0,24,36.0,12,16,12.0,0
5,144.0,72,128.0,50,100.0,64.0,24,45.0,12,16.0,4,4,1.5,0
6,144.0,72,128.0,50,100.0,64.0,24,45.0,12,16.0,4,4,1.5,0
7,144.0,72,128.0,50,100.0,64.0,24,45.0,12,16.0,4,4,1.5,0


## Revoke Pitch Normalization

In [31]:
for note in notes:
    note.pitch += root_offset

## Plott Piano Roll

In [32]:
output.pianoRoll(midi)

Loading BokehJS ...

In [33]:
output.synthesizeMidi(midi)